In [13]:
import os

import fiona
import geopandas as gpd
import gpxpy
import numpy as np
import pandas as pd
from multiprocess.pool import Pool
from tqdm.notebook import tqdm, trange
import warnings
import geopy.distance

warnings.filterwarnings('ignore')

fiona.drvsupport.supported_drivers['kml'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'

storage = "/Volumes/easystore/Drones/"

per_flight_block_seconds = pd.read_csv('../../data/outputs/outputs_drone-seconds-block-count-per-flight.csv',dtype=str)

flight_details = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"{storage}/gpx-with-census-data.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)

Loading data: 0it [00:00, ?it/s]

In [14]:
def build_paths(array_of_ids):
    compiled_flights =[] 
    for flight_id in array_of_ids:

        flight_path = gpd.read_file(f"/Volumes/easystore/Drones/flights/kml/{flight_id}.kml",driver='KML')
        flight_path['id'] = flight_id
        compiled_flights.append(flight_path)
    return pd.concat(compiled_flights)

In [15]:
geocodio_flights = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"{storage}/geocodio/all-flights-manifest_geocodio.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)
# a = pd.read_csv('../../data/outputs/outputs_cv-flight-manifest.csv')

Loading data: 0it [00:00, ?it/s]

In [16]:
geocodio_flights[(geocodio_flights['address_map'].astype(str).str.contains('1700',case=False))&(geocodio_flights['address_map'].astype(str).str.contains('broadway',case=False))]

,lat_map,lon_map,time,id,date,time_s,address_map,incident_id,type,department,...,Census Block Group,Full FIPS (block),Full FIPS (tract),Metro/Micro Statistical Area Name,Metro/Micro Statistical Area Code,Metro/Micro Statistical Area Type,Combined Statistical Area Name,Combined Statistical Area Code,Metropolitan Division Area Name,Metropolitan Division Area Code
1560,32.5930563443194,-117.074888979111,11:38am,58de5d94143d8527cc132af53a9aa87e,7-23-21,11:38am,1700 Broadway,NaN,Hot Team Assist,cvpd,...,1,060730132051017,06073013205,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN
2836,32.61736464,-117.07162882,1:46pm,ed246d91644ba93bcf07893f9c27a44d,9-14-21,1:46pm,1700 Broadway,CVL77376,Subject Causing Disturbance,cvpd,...,2,060730131022000,06073013102,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN
2837,32.6355209,-117.0731106,1:35pm,a9e795daf6e3f22f45aa3c98cd3f8315,9-14-21,1:35pm,1700 Broadway,CVL77376,Subject Causing Disturbance,cvpd,...,2,060730128002003,06073012800,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN
3139,32.6174123,-117.07167632,8:16am,29b1f6e179d9564bfac69b9a187c7ab6,1-26-22,8:16am,1700 Broadway,CVL7019,DUI,cvpd,...,2,060730131022000,06073013102,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN
6744,32.61742367,-117.07167327,2:44pm,9b0cdb6dbb05c3d58228ec011f4d59f1,9-27-22,2:44pm,1700 Broadway,CV2212713,Stolen vehicle,cvpd,...,2,060730131022000,06073013102,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN
10147,32.61740644,-117.07166125,10:59am,057d62256c1263cc8c35d72250787527,7-13-23,10:59am,1300 Third Av / 1700 Broadway,L059083/ L059088,Disorderly Person/ Impaired Driver,cvpd,...,2,060730131022000,06073013102,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN


In [ ]:
# pool = 060730123021012


060730127003001

In [55]:
lede = per_flight_block_seconds[(per_flight_block_seconds['GEOID20']=='060730132063003')]
flights = []
for f in lede['id'].drop_duplicates().values:
    flight = per_flight_block_seconds[per_flight_block_seconds['id']==f]
    flight = flight[flight['GEOID20']=='060730127003001']
    if flight.shape[0]>0:
        flights.append(flight)

,id,GEOID20,seconds,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,NAME20,MTFCC20,UR20,...,UATYPE20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,HOUSING20,POP20,geometry,len
40868,5db1d2295546e6d0d2726877c1c081e2,060730127003001,50,6,73,12700,3001,Block 3001,G5040,U,...,U,S,40518,0,32.6365542,-117.0848466,81,239,"POLYGON ((-117.087049 32.636423, -117.08691 32...",15


In [18]:
pp_paths = build_paths(['29b1f6e179d9564bfac69b9a187c7ab6'])
pp_paths = pd.merge(pp_paths,geocodio_flights,how='left',on='id')

pp_paths[['id','geometry','lat_map','lon_map','type']].to_csv('./public_pool.csv',index=False)
# pp_paths[['geometry','id']].drop_duplicates().to_csv('./low-paths.csv',index=False)

In [43]:
flight_details[flight_details['id']=='4342f9e237e48c34ddd15271a0ed1dc9'][['sequence','latitude','longitude','altitude']].to_csv('./flight.csv',index=False)

In [61]:
pp_paths

,Name,Description,geometry,id,lat_map,lon_map,time,date,time_s,address_map,...,Census Block Group,Full FIPS (block),Full FIPS (tract),Metro/Micro Statistical Area Name,Metro/Micro Statistical Area Code,Metro/Micro Statistical Area Type,Combined Statistical Area Name,Combined Statistical Area Code,Metropolitan Division Area Name,Metropolitan Division Area Code
0,Home,,POINT Z (-117.08270 32.63997 22.14233),5db1d2295546e6d0d2726877c1c081e2,32.6022197,-117.0692582,4:03pm,7-02-21,4:03pm,Fourth Ave./ Orange Ave.,...,3,060730132063003,06073013206,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN
1,Airdata.com,,"LINESTRING Z (-117.08270 32.63997 22.14233, -1...",5db1d2295546e6d0d2726877c1c081e2,32.6022197,-117.0692582,4:03pm,7-02-21,4:03pm,Fourth Ave./ Orange Ave.,...,3,060730132063003,06073013206,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN
